In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("20190410-Roche-LN-pt_table.csv", sep='\t')

In [4]:
df.head()

,Run,Region,Chain,MID_no,Primer,MID,Patient,Sample_Project,Reads,Species,Sample
0,run169,region1,BCRh,10,NaN,AGTCAGAGGC,LN40,LN-study,2500,human,10-IgA
1,run169,region2,BCRh,10,NaN,AGTCAGAGGC,LN75,LN-study,2500,human,10-IgA
2,run169,region1,BCRh,10,NaN,AGTCATGTCT,LN40,LN-study,2500,human,10-IgD
3,run169,region2,BCRh,10,NaN,AGTCATGTCT,LN75,LN-study,2500,human,10-IgD
4,run169,region1,BCRh,10,NaN,AGTCAGGGGA,LN40,LN-study,2500,human,10-IgG


In [13]:
ig = df["Sample"].tolist()
ig_new = list()
for i in range(len(ig)):
    if "-" in str(ig[i]):
        ig_new.append(ig[i].split("-")[1])
    else:
        ig_new.append('')
df["Sample"] = ig_new

In [14]:
df.head()

,Run,Region,Chain,MID_no,Primer,MID,Patient,Sample_Project,Reads,Species,Sample
0,run169,region1,BCRh,10,NaN,AGTCAGAGGC,LN40,LN-study,2500,human,IgA
1,run169,region2,BCRh,10,NaN,AGTCAGAGGC,LN75,LN-study,2500,human,IgA
2,run169,region1,BCRh,10,NaN,AGTCATGTCT,LN40,LN-study,2500,human,IgD
3,run169,region2,BCRh,10,NaN,AGTCATGTCT,LN75,LN-study,2500,human,IgD
4,run169,region1,BCRh,10,NaN,AGTCAGGGGA,LN40,LN-study,2500,human,IgG


In [15]:
patient = df["Patient"].tolist()
subtype = df["Sample"].tolist()

In [18]:
sample_name = list()
for i in range(len(patient)):
    if subtype[i] != '':
        sample_name.append(patient[i] + "-" + subtype[i])
    else:
        sample_name.append(patient[i])
df["Sample_Name"] = sample_name

In [19]:
df.head()

,Run,Region,Chain,MID_no,Primer,MID,Patient,Sample_Project,Reads,Species,Sample,Sample_Name
0,run169,region1,BCRh,10,NaN,AGTCAGAGGC,LN40,LN-study,2500,human,IgA,LN40-IgA
1,run169,region2,BCRh,10,NaN,AGTCAGAGGC,LN75,LN-study,2500,human,IgA,LN75-IgA
2,run169,region1,BCRh,10,NaN,AGTCATGTCT,LN40,LN-study,2500,human,IgD,LN40-IgD
3,run169,region2,BCRh,10,NaN,AGTCATGTCT,LN75,LN-study,2500,human,IgD,LN75-IgD
4,run169,region1,BCRh,10,NaN,AGTCAGGGGA,LN40,LN-study,2500,human,IgG,LN40-IgG


In [20]:
df.to_csv("20190410-Roche-LN-pt_table-reformatted.csv", index=False, sep=";")